In [1]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import torch.nn.functional as F
from sklearn.metrics import mean_squared_error
import seaborn as sns
import numpy as np
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import sklearn
from sklearn import preprocessing
import statistics 
from tqdm import tqdm
import copy
from random import randint 
import math
from matplotlib import ticker
from google.colab import files

ModuleNotFoundError: No module named 'torch'

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# %cd /content/drive/'My Drive'/'CS145_Kaggle'
# %ls


In [ ]:
traindf = pd.read_csv("data/train.csv")
testdf = pd.read_csv("data/test.csv", )
mobility = pd.read_csv("data/graph.csv")

In [ ]:
testdf.head()

### Fill-in test csv

In [ ]:
df1 = pd.read_csv("13.csv", usecols=["Province_State", "Confirmed", "Deaths"])
df1.tail()

In [ ]:
df2 = pd.read_csv("1.csv", usecols=["Province_State", "Confirmed", "Deaths"])
df2.tail()

In [ ]:
teststates = testdf.Province_State.unique().tolist()
newdfstates = df2.Province_State.unique().tolist()
skipstate = []
skipidx = []
idx = 0
for state in newdfstates:
  if state not in teststates:
    skipstate += [state]
    skipidx += [idx]
  idx += 1
skipstate

In [ ]:
df2 = df2.drop(index = skipidx)
df2 = df2.reset_index(drop=True)
print(len(df2))
df2.head()

In [ ]:
tempdf = copy.copy(testdf)
tempdf.iloc[0:1*50, 3:5] = df2[['Confirmed', 'Deaths']]

In [ ]:
for i in range(26):
  df = pd.read_csv("{}.csv".format(i+1), usecols=["Province_State", "Confirmed", "Deaths"])
  df = df.drop(index = skipidx)
  df = df.set_index(np.arange(i*50, (i+1)*50))
  assert len(df) == 50
  testdf.iloc[i*50:(i+1)*50, 3:5] = df[['Confirmed', 'Deaths']]
  

In [ ]:
# traintraindftttasdsadsadssssssstestdf

In [ ]:
from google.colab import files
testdf.to_csv('newtest.csv')  
files.download("newtest.csv")

### Data exploration

In [ ]:
print(traindf.shape)
traindf.head()

#### Translate states to numerical values (by category) 

In [ ]:
# translate Province_State to numerical values
unique_states = traindf.Province_State.unique()

In [ ]:
le = preprocessing.LabelEncoder() 
le.fit(unique_states)
le.classes_[:10]

In [ ]:
traindf["State_code"] = le.transform(traindf.Province_State)
features = traindf.columns.tolist() 
features = features[4:6] + features[0:2] + features[-1:] + features[2:4] + features[6:-1]
traindf = traindf[features]
traindf.iloc[48:52, :]

#### Check distribution and fill na

In [ ]:
# Any NA?
traindf.isnull().values.any()

In [ ]:
features = traindf.columns
features_na = []
for feature in features:
  na_count = traindf[feature].isnull().sum()
  if na_count > 0:
   features_na += [feature]
  print("{0: <20} : {1}".format(feature, na_count))

In [ ]:
# All states have NA in Hospitalization data (even after mid-June)
len(traindf[traindf["Hospitalization_Rate"].isna()]["Province_State"].unique())

In [ ]:
# Fill with zeros for now?
traindf['Hospitalization_Rate'] = traindf['Hospitalization_Rate'].fillna(0)
traindf['People_Hospitalized'] = traindf['People_Hospitalized'].fillna(0)

In [ ]:
traindf['Recovered'] = traindf['Recovered'].fillna(0)
traindf['Recovered'] 

In [ ]:
for feature in features_na:
  sns.displot(data=traindf, x=feature, kde=True)
  plt.show()

In [ ]:
traindf.corr()

In [ ]:
plt.figure(figsize=(8,8))
sns.heatmap(traindf.corr(),
            vmin=-1,
            cmap='coolwarm',
            annot=True)

In [ ]:
traindf.head()

In [ ]:
X_train = traindf.iloc[:, 4:]
y_train = traindf.iloc[:, 0:2]
X_train.shape, y_train.shape

### Models

### SVM RBF for baseline

In [ ]:
y_train["Deaths"].plot()

In [ ]:
y_train["Recovered"].plot(c = "orange")

In [ ]:
from statsmodels.tsa.ar_model import AutoReg

In [ ]:
import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from random import random
# contrived dataset
data = [x + random() for x in range(1, 100)]
# fit model
model = ARIMA(data, order=(1, 1, 1))
model_fit = model.fit()
# make prediction
yhat = model_fit.predict(len(data), len(data), typ='levels')
print(yhat)

In [ ]:
y_pred = y_train.rolling(2, win_type='gaussian').sum(std=3)

In [ ]:
y_train['D_SMA_10'] = y_train.Deaths.rolling(10, min_periods=1).mean()
y_train['D_SMA_20'] = y_train.Deaths.rolling(20, min_periods=1).mean()
y_train['R_SMA_10'] = y_train.Recovered.rolling(10, min_periods=1).mean()
y_train['R_SMA_20'] = y_train.Recovered.rolling(20, min_periods=1).mean()

In [ ]:
y_train["D_GSMA_10"] = y_train.Deaths.rolling(10, win_type='gaussian').sum(std=3)
y_train["D_GSMA_20"] = y_train.Deaths.rolling(20, win_type='gaussian').sum(std=3)

In [ ]:
y_train['DEMA_0.1'] = y_train.Deaths.ewm(alpha=0.1, adjust=False).mean()
y_train['DEMA_0.2'] = y_train.Deaths.ewm(alpha=0.2, adjust=False).mean()

In [ ]:
y_train

In [ ]:
pairs = [(2,3), (6,7), (8,9)]
for pair in pairs:
  y_train.iloc[:50,  [0,pair[0],pair[1]]].plot()
  plt.show()
  y_train.iloc[-50:, [0,pair[0],pair[1]]].plot()
  plt.show()

In [ ]:
#Other ideas
#use LSTM to train for predicting everything in the next day
#Arima 